# Mini Project: Temporal-Difference Methods

### Part 0: Explore CliffWalkingEnv

Use the code cell below to create an instance of the [CliffWalking](https://github.com/openai/gym/blob/master/gym/envs/toy_text/cliffwalking.py) environment.

In [ ]:
import gym
env = gym.make('CliffWalking-v0')

The agent moves through a $4\times 12$ gridworld, with states numbered as follows:
```
[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
 [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
 [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
 [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]]
```
At the start of any episode, state `36` is the initial state.  State `47` is the only terminal state, and the cliff corresponds to states `37` through `46`.

The agent has 4 potential actions:
```
UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3
```

Thus, $\mathcal{S}^+=\{0, 1, \ldots, 47\}$, and $\mathcal{A} =\{0, 1, 2, 3\}$.  Verify this by running the code cell below.

In [ ]:
print(env.action_space)
print(env.observation_space)

In this mini-project, we will build towards finding the optimal policy for the CliffWalking environment.  The optimal state-value function is visualized below.  Please take the time now to make sure that you understand _why_ this is the optimal state-value function.

In [ ]:
import numpy as np
from plot_utils import plot_values

# define the optimal state-value function
V_opt = np.zeros((4,12))
V_opt[0] = -np.arange(3, 15)[::-1]
V_opt[1] = -np.arange(3, 15)[::-1] + 1
V_opt[2] = -np.arange(3, 15)[::-1] + 2
V_opt[3][0] = -13

plot_values(V_opt)

### Part 1: TD Prediction: State Values

In this section, you will write your own implementation of TD prediction (for estimating the state-value function).

We will begin by investigating a policy where the agent moves:
- `RIGHT` in states `0` through `10`, inclusive,  
- `DOWN` in states `11`, `23`, and `35`, and
- `UP` in states `12` through `22`, inclusive, states `24` through `34`, inclusive, and state `36`.

The policy is specified and printed below.  Note that states where the agent does not choose an action have been marked with `-1`.

In [ ]:
policy = np.hstack([1*np.ones(11), 2, 0, np.zeros(10), 2, 0, np.zeros(10), 2, 0, -1*np.ones(11)])
print("\nPolicy (UP = 0, RIGHT = 1, DOWN = 2, LEFT = 3, N/A = -1):")
print(policy.reshape(4,12))

Run the next cell to visualize the state-value function that corresponds to this policy.  Make sure that you take the time to understand why this is the corresponding value function!

In [ ]:
V_true = np.zeros((4,12))
for i in range(3):
    V_true[0:12][i] = -np.arange(3, 15)[::-1] - i
V_true[1][11] = -2
V_true[2][11] = -1
V_true[3][0] = -17

plot_values(V_true)

The above figure is what you will try to approximate through the TD prediction algorithm.

Your algorithm for TD prediction has five arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `policy`: This is a 1D numpy array with `policy.shape` equal to the number of states (`env.nS`).  `policy[s]` returns the action that the agent chooses when in state `s`.
- `alpha`: This is the step-size parameter for the update step.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `V`: This is a dictionary where `V[s]` is the estimated value of state `s`.

Please complete the function in the code cell below.

In [ ]:
from collections import defaultdict, deque
import sys

def td_prediction(env, num_episodes, policy, alpha, gamma=1.0):
    # initialize empty dictionaries of floats
    V = defaultdict(float)
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 100 == 0:
            print("\rEpisode {}/{}".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        
        state = env.reset()
        
        while True:
            action = policy[state]
            
            next_state, reward, done, info = env.step(action)
            
            V[state] += alpha*(reward + gamma * V[next_state] - V[state])
            state = next_state
            
            if done:
                break
        
    return V 

Run the code cell below to test your implementation and visualize the estimated state-value function.  If the code cell returns **PASSED**, then you have implemented the function correctly!  Feel free to change the `num_episodes` and `alpha` parameters that are supplied to the function.  However, if you'd like to ensure the accuracy of the unit test, please do not change the value of `gamma` from the default.

In [ ]:
import check_test

# evaluate the policy and reshape the state-value function
V_pred = td_prediction(env, 5000, policy, .01)

# please do not change the code below this line
V_pred_plot = np.reshape([V_pred[key] if key in V_pred else 0 for key in np.arange(48)], (4,12)) 
check_test.run_check('td_prediction_check', V_pred_plot)
plot_values(V_pred_plot)

How close is your estimated state-value function to the true state-value function corresponding to the policy?  

You might notice that some of the state values are not estimated by the agent.  This is because under this policy, the agent will not visit all of the states.  In the TD prediction algorithm, the agent can only estimate the values corresponding to states that are visited.

### Part 2: TD Control: Sarsa

In this section, you will write your own implementation of the Sarsa control algorithm.

Your algorithm has four arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `alpha`: This is the step-size parameter for the update step.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `Q`: This is a dictionary (of one-dimensional arrays) where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.

Please complete the function in the code cell below.

(_Feel free to define additional functions to help you to organize your code._)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def generate_micro_policy(q_s, e, n_actions):
    if np.array_equal(q_s, np.zeros(n_actions)):
        return np.ones(n_actions) * 1/n_actions  
    
    micro_policy_s = np.ones(n_actions) * [e/n_actions]
    best_action = np.argmax(q_s)
    micro_policy_s[best_action] = 1 - e + e/n_actions
    
    return micro_policy_s
    
def take_an_action(Q, state, e, env):
    n_actions = env.nA
    policy = generate_micro_policy(Q[state], e, n_actions)
    
    return np.random.choice(np.arange(n_actions),
                            p=policy)

def sarsa(env, num_episodes, alpha, gamma=1.0):
    # initialize action-value function (empty dictionary of arrays)
    Q = defaultdict(lambda: np.zeros(env.nA))
    # initialize performance monitor
    # loop over episodes
    
    scores = []
    actions = []
    
    plot_every = 25
    
    deque_scores = deque(maxlen=plot_every)
    deque_actions = deque(maxlen=plot_every)
    
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 100 == 0:
            print("\rEpisode {}/{}".format(i_episode, num_episodes), end="")
            sys.stdout.flush()   
        
        e = 1.0/i_episode
        
        state = env.reset()
        action = take_an_action(Q, state, e, env)
        
        #create an score for further analysis
        score_in_episode = 0
        
        #max od 300 actions per episode
        for actions_made in range(300):
            next_state, reward, done, info = env.step(action)
            next_action = take_an_action(Q, next_state, e, env)
            
            Q[state][action] += alpha*(reward
                                       + gamma * Q[next_state][next_action]
                                       - Q[state][action])
            
            state = next_state
            action = next_action
            
            score_in_episode += reward
            
            if done:
                break
                
        #append score for further plot
        deque_scores.append(score_in_episode)
        deque_actions.append(actions_made)
        
        if (i_episode % plot_every == 0):
            scores.append(np.mean(deque_scores))
            actions.append(np.mean(deque_actions))
                
    # plot performance
    plt.plot(np.linspace(0, num_episodes, len(scores), endpoint=False),
             np.asarray(scores))
    plt.ylabel('Average Reward (Over Next %d Episodes)' % plot_every)
    plt.xlabel('Episode Number')
    plt.show()
    
    plt.plot(np.linspace(0, num_episodes, len(actions), endpoint=False),
             np.asarray(actions))
    plt.xlabel('Episode Number')
    plt.ylabel('Average Actions made (Over Next %d Episodes)' % plot_every)
    plt.show()
    
    return Q

Use the next code cell to visualize the **_estimated_** optimal policy and the corresponding state-value function.  

If the code cell returns **PASSED**, then you have implemented the function correctly!  Feel free to change the `num_episodes` and `alpha` parameters that are supplied to the function.  However, if you'd like to ensure the accuracy of the unit test, please do not change the value of `gamma` from the default.

In [ ]:
# obtain the estimated optimal policy and corresponding action-value function
Q_sarsa = sarsa(env, 5000, .01)

# print the estimated optimal policy
policy_sarsa = np.array([np.argmax(Q_sarsa[key]) if key in Q_sarsa else -1 for key in np.arange(48)]).reshape(4,12)
check_test.run_check('td_control_check', policy_sarsa)
print("\nEstimated Optimal Policy (UP = 0, RIGHT = 1, DOWN = 2, LEFT = 3, N/A = -1):")
print(policy_sarsa)

# plot the estimated optimal state-value function
V_sarsa = ([np.max(Q_sarsa[key]) if key in Q_sarsa else 0 for key in np.arange(48)])
plot_values(V_sarsa)

### Part 3: TD Control: Q-learning

In this section, you will write your own implementation of the Q-learning control algorithm.

Your algorithm has four arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `alpha`: This is the step-size parameter for the update step.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `Q`: This is a dictionary (of one-dimensional arrays) where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.

Please complete the function in the code cell below.

(_Feel free to define additional functions to help you to organize your code._)

In [ ]:
#q-learning
import matplotlib.pyplot as plt
%matplotlib inline

def q_learning(env, num_episodes, alpha, gamma=1.0):
    Q = defaultdict(lambda: np.zeros(env.nA))
    
    scores = []
    actions = []
    
    plot_every = 25
    
    deque_scores = deque(maxlen=plot_every)
    deque_actions = deque(maxlen=plot_every)

    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 100 == 0:
            print("\rEpisode {}/{}".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        
        ## TODO: complete the function 
        
        e = 1/i_episode
        
        state = env.reset()
        action = take_an_action(Q, state, e, env)
        
        #create an score for further analysis
        score_in_episode = 0
        actions_made = 0
        
        #max od 300 actions per episode
        for actions_made in range(300):
            action = take_an_action(Q, state, e, env)
            next_state, reward, done, info = env.step(action)
            
            Q[state][action] += alpha*(reward 
                                       + gamma * np.max(Q[next_state])
                                       - Q[state][action])
            
            state = next_state
            
            score_in_episode += reward
            
            if done:
                break
                
        #append score for further plot
        deque_scores.append(score_in_episode)
        deque_actions.append(actions_made)
        
        if (i_episode % plot_every == 0):
            scores.append(np.mean(deque_scores))
            actions.append(np.mean(deque_actions))
                
    # plot performance
    plt.plot(np.linspace(0, num_episodes, len(scores), endpoint=False),
             np.asarray(scores))
    plt.ylabel('Average Reward (Over Next %d Episodes)' % plot_every)
    plt.xlabel('Episode Number')
    plt.show()
    
    plt.plot(np.linspace(0, num_episodes, len(actions), endpoint=False),
             np.asarray(actions))
    plt.xlabel('Episode Number')
    plt.ylabel('Average Actions made (Over Next %d Episodes)' % plot_every)
    plt.show()
    
    return Q

Use the next code cell to visualize the **_estimated_** optimal policy and the corresponding state-value function. 

If the code cell returns **PASSED**, then you have implemented the function correctly!  Feel free to change the `num_episodes` and `alpha` parameters that are supplied to the function.  However, if you'd like to ensure the accuracy of the unit test, please do not change the value of `gamma` from the default.

In [ ]:
# obtain the estimated optimal policy and corresponding action-value function
Q_sarsamax = q_learning(env, 5000, .01)

# print the estimated optimal policy
policy_sarsamax = np.array([np.argmax(Q_sarsamax[key]) if key in Q_sarsamax else -1 for key in np.arange(48)]).reshape((4,12))
check_test.run_check('td_control_check', policy_sarsamax)
print("\nEstimated Optimal Policy (UP = 0, RIGHT = 1, DOWN = 2, LEFT = 3, N/A = -1):")
print(policy_sarsamax)

# plot the estimated optimal state-value function
plot_values([np.max(Q_sarsamax[key]) if key in Q_sarsamax else 0 for key in np.arange(48)])

### Part 4: TD Control: Expected Sarsa

In this section, you will write your own implementation of the Expected Sarsa control algorithm.

Your algorithm has four arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `alpha`: This is the step-size parameter for the update step.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `Q`: This is a dictionary (of one-dimensional arrays) where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.

Please complete the function in the code cell below.

(_Feel free to define additional functions to help you to organize your code._)

In [ ]:
def expected_sarsa(env, num_episodes, alpha, gamma=1.0):
    # initialize empty dictionary of arrays
    Q = defaultdict(lambda: np.zeros(env.nA))
    
    scores = []
    actions = []
    
    plot_every = 25
    
    deque_scores = deque(maxlen=plot_every)
    deque_actions = deque(maxlen=plot_every)

    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 100 == 0:
            print("\rEpisode {}/{}".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
            
        ## s very small epsilon
        e = 1 / (num_episodes * 1000)
        
        state = env.reset()
        action = take_an_action(Q, state, e, env)
        
        #create an score for further analysis
        score_in_episode = 0
        actions_made = 0
        
        #max od 300 actions per episode
        for actions_made in range(300):
            action = take_an_action(Q, state, e, env)
            next_state, reward, done, info = env.step(action)
            
            policy = generate_micro_policy(Q[next_state], e, env.nA)
            
            Q[state][action] += alpha*(reward 
                                       + gamma * policy.dot(Q[next_state])
                                       - Q[state][action])
            
            state = next_state
            
            score_in_episode += reward
            
            if done:
                break
                
        #append score for further plot
        deque_scores.append(score_in_episode)
        deque_actions.append(actions_made)
        
        if (i_episode % plot_every == 0):
            scores.append(np.mean(deque_scores))
            actions.append(np.mean(deque_actions))
                
    # plot performance
    plt.plot(np.linspace(0, num_episodes, len(scores), endpoint=False),
             np.asarray(scores))
    plt.ylabel('Average Reward (Over Next %d Episodes)' % plot_every)
    plt.xlabel('Episode Number')
    plt.show()
    
    plt.plot(np.linspace(0, num_episodes, len(actions), endpoint=False),
             np.asarray(actions))
    plt.xlabel('Episode Number')
    plt.ylabel('Average Actions made (Over Next %d Episodes)' % plot_every)
    plt.show()
    
    return Q

Use the next code cell to visualize the **_estimated_** optimal policy and the corresponding state-value function.  

If the code cell returns **PASSED**, then you have implemented the function correctly!  Feel free to change the `num_episodes` and `alpha` parameters that are supplied to the function.  However, if you'd like to ensure the accuracy of the unit test, please do not change the value of `gamma` from the default.

In [ ]:
# obtain the estimated optimal policy and corresponding action-value function
Q_expsarsa = expected_sarsa(env, 5000, 1)

# print the estimated optimal policy
policy_expsarsa = np.array([np.argmax(Q_expsarsa[key]) if key in Q_expsarsa else -1 for key in np.arange(48)]).reshape(4,12)
check_test.run_check('td_control_check', policy_expsarsa)
print("\nEstimated Optimal Policy (UP = 0, RIGHT = 1, DOWN = 2, LEFT = 3, N/A = -1):")
print(policy_expsarsa)

# plot the estimated optimal state-value function
plot_values([np.max(Q_expsarsa[key]) if key in Q_expsarsa else 0 for key in np.arange(48)])